# Computing the PFSS model
***

In this example, we see how to compute a PFSS coronal magnetic field and do basic operations with it.

The routines related to the coronal magnetic field are located in the `euhforia/corona/magnetic_field` directory. To access it, the path of the EUHFORIA root directory needs to be included. In addition, we import a few basic Python modules that we will be needing.

In [ ]:
import numpy as np
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import euhforia.core.constants as constants

In [ ]:
Rs = constants.solar_radius

Let's retrieve a GONG/ADAPT magnetogram for a specific date (check the notebooks in the `examples/magnetogram` directory for more examples on working with magnetograms)

In [ ]:
date = datetime.datetime(2012, 6, 30, 2) # year, month, day, hour

In [ ]:
import euhforia.magnetogram.gong_adapt

Download the magnetogram 

In [ ]:
adapt_magnetogram_file = euhforia.magnetogram.gong_adapt.retrieve(date, save_directory="data/")

Load it

In [ ]:
adapt_magnetogram = euhforia.magnetogram.gong_adapt.read(adapt_magnetogram_file, realization_idx=0)

To compute the PFSS model, the `euhforia.corona.magnetic_field.pfss` module is used.

In [ ]:
import euhforia.corona.magnetic_field.pfss

In [ ]:
pfss = euhforia.corona.magnetic_field.pfss.PFSS(max_degree=140)

In [ ]:
# You can view the source documentation using the help() function

#help(pfss)

Next, we set the magnetogram that will be used as boundary condition:

In [ ]:
pfss.magnetogram = adapt_magnetogram

Finally, the model is computed (this will take a couple of minutes depending on `max_degree`)

In [ ]:
pfss.compute()

Let's compute the resulting magnetic field at the inner boundary:

In [ ]:
Br, Bt, Bp = pfss.get_synoptic_map(Rs)

Now that we have the components, we can plot them:

In [ ]:
import euhforia.plot.synoptic

In [ ]:
Br.plot_settings['norm'] = plt.Normalize(-100, 100)
euhforia.plot.synoptic.plot(Br);

Let's plot the original magnetogram

In [ ]:
adapt_magnetogram.plot_settings['norm'] = plt.Normalize(-100, 100)
euhforia.plot.synoptic.plot(adapt_magnetogram);

Let's compute the magnetic field higher up in the corona

In [ ]:
Br, Bt, Bp = pfss.get_synoptic_map(2.5*Rs)

Now, plotting the radial component with appropriate thresholds shows the heliospheric current sheet

In [ ]:
Br.plot_settings['norm'] = plt.Normalize(-0.0001, 0.0001)
euhforia.plot.synoptic.plot(Br);

We can also compute the PFSS solution on a grid as required by e.g. the field line tracer

In [ ]:
field = pfss.compute_field_on_grid(np.linspace(1.0, 2.5, 120)*Rs)

The gridded output can be saved e.g. to VTK for external visualization:

In [ ]:
import pyevtk.hl as evtk

In [ ]:
evtk.gridToVTK(pfss.magnetogram.date.isoformat()+"_pfss",
               field.grid.axis.r.coordinates/constants.solar_radius,
               field.grid.axis.clt.coordinates,
               field.grid.axis.lon.coordinates,
               pointData={"Br" : field.data["Br"], "Bt" : field.data["Bt"], "Bp" : field.data["Bp"]}
               )

## Field line tracing

Tracing field lines is often of interest. This can be done using the `euhforia.tools.field_line_tracer` module

In [ ]:
import euhforia.tools.field_line_tracer

Define the method that performs the tracing:

In [ ]:
tracer = euhforia.tools.field_line_tracer.FieldlineTracer(field, method="rk2")

Of particular interest is to trace a bunch of field lines in order to get a global view of the topology. For this, there is the `SynopticFieldLineTracer` - class. Let's trace field lines from the lower boundary upwards:

In [ ]:
trace_from_photosphere \
    = euhforia.tools.field_line_tracer.SynopticFieldLineTracer(radius=1.005*Rs, 
                                                               deg_per_pixel=2.0, 
                                                               max_path_length=5.0*Rs,
                                                               min_step_size=0.001*Rs,
                                                               direction="outward", 
                                                               tracer=tracer,
                                                               field_model=pfss, 
                                                               date=pfss.magnetogram.date,
                                                               crval1=pfss.magnetogram.meta['crval1'])

Computing the field lines takes a while...

In [ ]:
%time trace_from_photosphere.compute()

Now we can e.g. look where the field lines ended up:

In [ ]:
euhforia.plot.synoptic.plot(trace_from_photosphere.at_end_pt["r"]);

With this informatio, we can compute the open and closed field regions:

In [ ]:
import euhforia.corona.magnetic_field.topology as topology

In [ ]:
open_closed_regions = topology.open_closed.compute_open_and_closed_fields(trace_from_photosphere, 2.49*Rs)

In [ ]:
open_closed_regions.plot_settings["cmap"] = "jet"
fig, ax = euhforia.plot.synoptic.plot(open_closed_regions)

Often it is also interesting to trace the field lines starting from higher up in the corona:

In [ ]:
trace_from_above \
    = euhforia.tools.field_line_tracer.SynopticFieldLineTracer(radius=2.4*Rs, 
                                                               deg_per_pixel=2.0, 
                                                               max_path_length=5.0*Rs, 
                                                               direction="inward",
                                                               min_step_size=0.001*Rs,
                                                               tracer=tracer,
                                                               field_model=pfss,
                                                               date=pfss.magnetogram.date,
                                                               crval1=pfss.magnetogram.meta['crval1'])
trace_from_above.compute()

This allows us to compute the flux tube expansion factor:

In [ ]:
fte_map = topology.fte.compute_flux_tube_expansion_map(trace_from_above)

In [ ]:
fte_map.plot_settings["norm"] = plt.Normalize(0, 400)
fte_map.plot_settings["cmap"] = "Spectral_r"
euhforia.plot.synoptic.plot(fte_map);

***